In [3]:
from tqdm import tqdm
import datetime
tqdm.pandas()

import pandas as pd
import numpy as np
#connect local PATH
from matplotlib import pyplot as plt

import os 
import sys
p = os.path.abspath(r'C:\Users\Ben\Desktop\Diplomatiki')
sys.path.append(str(p))
from CryptoSent.tools.crypto_coin_actual import get_btc_actual_hourly
from CryptoSent.tools import evaluation as eval
#from CryptoSent.tools.utils import load_submissions

from sklearn.preprocessing import MinMaxScaler

def errors(x,Y):
    return [Y[i]-x[i] for i in range(len(x))]

def abolute_errors(x,Y):
    return [abs(Y[i]-x[i]) for i in range(len(x))]

    
def mean_error(x,Y):
    return np.mean([Y[i]-x[i] for i in range(len(x))])

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score  
from sklearn.metrics import mean_squared_error as mse   #squared=False:rmse
#from sklearn.metrics import mean_squared_log_error as msle      #squared=False:rmsle

from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model
from sklearn.linear_model import TweedieRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

import itertools

evaluation_metrics=[mean_error,mae,mape,mse,r2_score]
evaluation_metrics_names=['me','mae','mape','mse','r2_score','rmse']
def evaluate_pred(x,Y,model_name='m1'):
    scores=[]
    for metric in evaluation_metrics:
        scores.append(metric(x,Y))
    scores.append(mse(x,Y,squared=False))
    output=pd.DataFrame(
        np.array(scores),
        index=evaluation_metrics_names,
        columns=[model_name])
    return output



# import datasets
#subs=load_submissions()
btc=get_btc_actual_hourly().sort_index()
btc['perc']=btc.close.pct_change()
btc['difference']=btc.close.diff()
#btc=btc[['close','perc','difference']]
btc=btc.dropna()



#bc=pd.read_pickle("../Datasets/Main Dataset/bitcoin_comments.pickle")
subs=pd.read_pickle("../../Datasets/Main Dataset/bitcoin_submissions.pickle")
subs['volume']=1
#bc['buy']=bc.body_clean.str.contains('buy',case=False)
#bc['sell']=bc.body_clean.str.contains('sell',case=False)
#bc['volume']=1

btc_raw=get_btc_actual_hourly()

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, ExponentialSmoothing, BlockRNNModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

In [5]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scl=StandardScaler()
sclb=StandardScaler()
mmsclzo=MinMaxScaler(feature_range=(0,1))

resample_period='d' ######################
TARGET='close'   #########################

#filter df
sentiment_signals=[
'volume','sell', 'buy', 
'sent_ntusd','sent_ntusd_wavg','sent_ntusd_coms','sent_ntusd_coms_wavg', 
'sent_lr','sent_lr_coms', 
'sent_db', 'sent_fb', 'sent_fbt', 
'sent_dbe_sadness','sent_dbe_joy', 'sent_dbe_love',
'sent_dbe_anger', 'sent_dbe_fear','sent_dbe_surprise'
]

#filter on sentiment signals, topic and any other
v_topic=subs.topic2=='Bitcoin'
v_score=subs.origin!='main'

v_awards=subs.awards_value > subs.awards_value.quantile(.99)
v_awards_resample=subs.resample(resample_period).awards_value.transform(lambda x: x>x.quantile(.90))

v_karma=subs.author_karma > subs.author_karma.quantile(.90)
v_karma_resample=subs.resample(resample_period).author_karma.transform(lambda x: x>x.quantile(.90))


aggtactic='sum'
filtered_df=subs[v_topic][sentiment_signals].drop(columns=['volume']).resample(resample_period).sum()#### AGGREGATION TACTIC

# & (v_score |v_awards |v_karma)
#filter on btc features
btc_fts=['unix', 'date', 'symbol', 'open','close', 'high', 'low', 'close', 'Volume BTC',
       'Volume USD', 'avg_hl', 'avg_oc', 'perc', 'difference']
btc_filtered=btc.drop(columns=['unix','date','symbol','avg_hl','avg_oc'])



scaled__target=pd.DataFrame(sclb.fit_transform(btc_filtered[TARGET].to_numpy().reshape(-1,1)),
             index=btc_filtered[TARGET].index,
             columns=[TARGET]).resample(resample_period).mean()


tempflt=pd.DataFrame(
    scl.fit_transform(filtered_df),
    columns=filtered_df.columns,
    index=filtered_df.index)
#tempflt=tempflt.resample(resample_period).sum()


maindf=pd.concat([
    btc_filtered.drop(columns=TARGET).resample(resample_period).mean(),
    #filtered_df.resample(resample_period).mean(),
    tempflt,
    subs[v_topic].resample(resample_period).volume.sum()],axis=1).dropna().head(-1)



dft=pd.DataFrame(
    scl.fit_transform(maindf),
    columns=maindf.columns,
    index=maindf.index).dropna()

dft=pd.concat([dft,scaled__target],axis=1).dropna()

#dft=pd.DataFrame(
#    scl.fit_transform(maindf.drop(columns=[TARGET])),
#    columns=maindf.drop(columns=[TARGET]).columns,
#    index=maindf.drop(columns=[TARGET]).index)[:'2021-06']


target=dft[TARGET]

##################################################################################
#features=dft.drop(columns=[TARGET])

#features=dft.drop(columns=['open'])
#features=dft[['open','high','low','sent_dbe_joy']]
features=dft[sentiment_signals]#['high','difference','buy','sent_lr_coms','volume']]
#features=dft[['close','high','difference','buy','sent_lr_coms','volume']]
#features=dft[['sent_dbe_joy']]
#features=dft[['open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD', 'perc','difference']]
#features=dft[['high', 'low', 'close']]
######################################################################
dt_index=dft.index
featuresnp=features.to_numpy()
targetnp=target.to_numpy()

count=0
train_window=5 #####################
test_window=1
test_lenght=30*1*1 #####################
x=[]
y=[]
for i in range(target.shape[0]-train_window-test_window):
    x.append(featuresnp[i:train_window+i])
    y.append(targetnp[train_window+i:train_window+i+test_window])

x=np.array(x)
y=np.array(y)
x=x.reshape(x.shape[0],x.shape[1]*x.shape[2])
y=y.reshape(y.shape[0],)
xt,xv=x[:-test_lenght],x[-test_lenght:]
yt,yv=y[:-test_lenght],y[-test_lenght:]

In [41]:
resample_period='d'
series_start='2019-01-20'
series_end='2021-06-30'
split_date='20210531'

s1=subs.sent_db.resample(resample_period).sum()
s2=subs.sent_fb.resample(resample_period).sum()
s3=subs.sent_lr.resample(resample_period).sum()
s4=subs.sent_ntusd.resample(resample_period).sum()
s5=subs.sent_fbt.resample(resample_period).sum()
s6=subs.buy.resample(resample_period).sum()
s7=subs.sell.resample(resample_period).sum()
s8=subs.volume.resample(resample_period).sum()
s9=subs.sent_ntusd_coms.resample(resample_period).sum()
#s10=subs.buy_sell.resample(resample_period).sum()


btcin=btc.resample(resample_period).mean().dropna()

a=pd.concat([s1,s2,s3,s4,s5,s6,s7,s8,s9,btcin],axis=1)
a=a[~a.sent_fbt.isna()].head(-1)
#a=a['2021-01-15':'2021-02-13']
a=a[series_start:series_end]

# Create a TimeSeries, specifying the time and value columns
series = TimeSeries.from_dataframe(a.reset_index(), 'index', 'close')
# Create training and validation sets:
train, val = series.split_after(pd.Timestamp(split_date))
future_prediction_period=len(val)

transformer = Scaler()
train_transformed = transformer.fit_transform(train)
val_transformed = transformer.transform(val)
series_transformed = transformer.transform(series)


c1=Scaler().fit_transform(TimeSeries.from_dataframe(a.reset_index(), 'index', 'volume'))


#c1=Scaler().fit_transform(TimeSeries.from_dataframe(a.reset_index(), 'index', 'sell'))
#c2=Scaler().fit_transform(TimeSeries.from_dataframe(a.reset_index(), 'index', 'sent_db'))
#c3=Scaler().fit_transform(TimeSeries.from_dataframe(a.reset_index(), 'index', 'sent_lr'))
#c4=Scaler().fit_transform(TimeSeries.from_dataframe(a.reset_index(), 'index', 'sent_fbt'))

covariates = c1#.stack(c2).stack(c3).stack(c4)
cov_train, cov_val = covariates.split_after(pd.Timestamp(split_date))





In [42]:
my_model = RNNModel(
    model='LSTM',
    n_rnn_layers=2, 
    hidden_dim=20,
    dropout=0.2,
    batch_size=32,
    n_epochs=50,
    optimizer_kwargs={'lr': 0.001}, 
    model_name='Air_RNN',
    log_tensorboard=True,
    random_state=42,
    training_length=5,
    input_chunk_length=12,
    force_reset=True,
    save_checkpoints=True
)


In [50]:
my_model.fit(train_transformed, 
#             future_covariates=covariates, 
#            val_series=val_transformed, 
#             val_future_covariates=covariates, 
             verbose=True)


ValueError: __len__() should return >= 0

In [52]:
my_model.predict(3)

ValueError: This model has been trained without covariates. No covariates should be provided for prediction.

In [46]:
def eval_model(model):
    pred_series = model.predict(n=future_prediction_period, future_covariates=covariates)
    plt.figure(figsize=(8,5))
    #series_transformed.plot(label='actual')
    series_transformed[-len(pred_series):].plot()
    pred_series.plot(label='forecast')
    plt.title('MAPE: {:.2f}%'.format(mape(pred_series, val_transformed)))
    plt.legend();


eval_model(my_model)

ValueError: This model has been trained without covariates. No covariates should be provided for prediction.

In [6]:
my_model = RNNModel(
    model="LSTM",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=300,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    save_checkpoints=True,
)

In [7]:
my_model.fit(
    xt,

    val_series=yt,

    verbose=True,
)

AttributeError: 'numpy.ndarray' object has no attribute 'width'